In [1]:
import os
import datetime
import numpy as np
import pandas as pd

In [13]:
from datasets import load_dataset
import os
from PIL import Image

data_folder = "../../data/nonfixed_data/"
dataset = load_dataset(data_folder)

Resolving data files:   0%|          | 0/18900 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/3500 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2100 [00:00<?, ?it/s]

Found cached dataset imagefolder (/home/k6zma/.cache/huggingface/datasets/imagefolder/nonfixed_data-ea9b51fcd96792cd/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [15]:
from transformers import AutoImageProcessor

checkpoint = 'checkpoint-3600'
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [16]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [17]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

In [18]:
dataset = dataset.with_transform(transforms)

In [19]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [20]:
import evaluate

accuracy = evaluate.load("accuracy")

In [21]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [11]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)
model = model.to("cuda")

In [12]:
training_args = TrainingArguments(
    output_dir="my_model",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=100,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,0.011700,0.813082,0.836000


KeyError: 'eval_valid_acc'

In [22]:
ds = load_dataset(data_folder)

Resolving data files:   0%|          | 0/18900 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/3500 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2100 [00:00<?, ?it/s]

Found cached dataset imagefolder (/home/k6zma/.cache/huggingface/datasets/imagefolder/nonfixed_data-ea9b51fcd96792cd/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [24]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch
from tqdm import tqdm

model = AutoModelForImageClassification.from_pretrained('checkpoint-3600')
model = model.to("cuda")
pred = []
label = []
image = []
image_processor = AutoImageProcessor.from_pretrained("checkpoint-3600")
for i in tqdm(range(len(ds["test"]["image"]))):
    image = ds["test"]["image"][i]
    inputs = image_processor(image, return_tensors="pt")
    inputs = inputs.to("cuda")
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_label = logits.argmax(-1).item()
    pred.append(predicted_label)
    label.append(ds["test"]["label"][i])

In [9]:
print(pred)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [11]:
print(label)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [21]:
k = 0
j = 0
for j in tqdm(range(len(pred))):
    if label[j] == pred[j]:
        k += 1
    else:
        print(pred[j], j, '- неправильно угадал')
res = (k * 100) / len(label)
print(res)

100%|██████████| 3500/3500 [00:00<00:00, 260839.80it/s]

5 591 - неправильно угадал
2 593 - неправильно угадал
2 594 - неправильно угадал
2 608 - неправильно угадал
3 609 - неправильно угадал
2 610 - неправильно угадал
2 612 - неправильно угадал
2 614 - неправильно угадал
2 615 - неправильно угадал
2 616 - неправильно угадал
2 618 - неправильно угадал
2 619 - неправильно угадал
2 629 - неправильно угадал
2 630 - неправильно угадал
2 631 - неправильно угадал
2 632 - неправильно угадал
2 636 - неправильно угадал
2 638 - неправильно угадал
2 640 - неправильно угадал
2 641 - неправильно угадал
2 645 - неправильно угадал
2 646 - неправильно угадал
2 652 - неправильно угадал
0 758 - неправильно угадал
0 759 - неправильно угадал
0 855 - неправильно угадал
2 1538 - неправильно угадал
2 1539 - неправильно угадал
1 1542 - неправильно угадал
5 1543 - неправильно угадал
5 1544 - неправильно угадал
2 1545 - неправильно угадал
2 1546 - неправильно угадал
2 1547 - неправильно угадал
2 1548 - неправильно угадал
5 1549 - неправильно угадал
5 1550 - неправиль